In [1]:
## Imports 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from pprint import pprint 
import warnings
from collections import deque
warnings.filterwarnings("ignore")
import copy

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

<b style="color:darkorange">Example 1 : </b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

We use the following martingales to get the upper bound :

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

Reference price of the option is $5.152$.

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in the example. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [3]:
Bs = np.array([ 87.52900166,  87.80954309,  88.09098369,  88.37332635, 88.72752759,  89.1544435 ,  89.61925406,  90.19465661, 90.88274744,  91.75942312,  92.97867688,  94.85697075, 100.])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [4]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)                      

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation


paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in range(npaths):
        if exer_or_cont(i, paths[i][j]):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1408
Dual Price   (Upper Bound) = 5.1474


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

----------------------------------------------------------------------------------------------------------------------------

$M_t\equiv0$.


In [5]:
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=100000)
hedges = np.zeros(paths.shape, dtype=np.float)
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1408
Dual Price   (Upper Bound) = 8.5835


$M_t$ is the discounted European put price with the same final maturity less the initial price.

In [6]:
hedges = [ blackscholes_price(K, ts[-1]-ts[i],paths[i],vol,r, q,'put')*np.exp(-ts[i]*r) for i in range(len(ts))]
hedges = [hedges[i]-hedges[0] for i in range(len(hedges))]
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1408
Dual Price   (Upper Bound) = 5.3466


The upper bounds are inline with our expectation because we see that when the position is unhedged we will have a higher upperbound as compared to the case when the position is hedged

<b style="color:darkorange">Example 2 :</b> Consider pricing the Bermudan put option as previous example by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. We use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then use the Broadie-Andersen algorithm to compute the corresponding upper bound.


In [7]:
## Longstaff-Schwartz Algorithm 
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=np.float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
print("Estimates :",betas_LS)                                           # No regression needed at first and last time steps, return NaN

V0 = nested_mc(S0,vol, r, q, 0, ts, 1000000)


Estimates : [[        nan         nan]
 [-0.49208962  1.30361804]
 [-0.55495172  1.30597871]
 [-0.47764151  1.2833809 ]
 [-0.36527465  1.24711654]
 [-0.315787    1.21733515]
 [-0.16702099  1.15196147]
 [-0.13433194  1.13849548]
 [-0.03690334  1.0974072 ]
 [-0.02028372  1.07279482]
 [ 0.02139218  1.0431469 ]
 [ 0.00274591  1.00265275]
 [        nan         nan]]


We now define a new function now to make exercise decision (Longstaff-Schwartz algorithm) 

In [8]:
def exer_or_cont_LS(i,S,K,Z,beta):
    contval = beta[i,0] + beta[i,1]*Z
    exerval = np.maximum(K-S,0) 
    return  exerval >= contval

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):          # new nested_mc that uses estimated values for exercise decision
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        Z = blackscholes_price(K, ts[-1]-ts[i], nested_paths, vol, r, q, callput='put')
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont_LS(j, nested_paths,K,Z,betas_LS)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

In [9]:
# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i]) 
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    ind = exer_or_cont_LS(i,paths[i],K,Z,betas_LS)                                # True for exercise False for continue
    for j in range(npaths):
        if exer_or_cont_LS(i,paths[i][j],K,Z[j],betas_LS):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1436
Dual Price   (Upper Bound) = 5.1762


<b style="color:darkorange">Example 3 :</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, we use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.


In [10]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=np.float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
print(betas_TVR)                                                 # No regression needed at first and last time steps, return NaN

V0 = np.mean(V)*np.exp(-r*(ts[1]-ts[0]))


[[        nan         nan]
 [-0.46564688  1.3160679 ]
 [-0.38767136  1.29401702]
 [-0.31683182  1.26878016]
 [-0.23145839  1.2340282 ]
 [-0.16752664  1.20323228]
 [-0.10875014  1.16465352]
 [-0.04808141  1.12914112]
 [-0.01907371  1.09760004]
 [ 0.01803298  1.06180154]
 [ 0.01563988  1.03028454]
 [ 0.00214919  0.99894957]
 [        nan         nan]]


In [11]:
def nested_mc_TVR(S, vol, r, q, i, ts, nnested,beta) :
    nested_paths = np.full(nnested, S, dtype=np.float)
    dt = ts[i+1]-ts[i]
    dW = np.random.randn(len(nested_paths))*np.sqrt(dt)
    nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
    Z = blackscholes_price(K, ts[-1]-ts[i+1], nested_paths, vol, r, q, callput='put')
    exerval = np.maximum(K-nested_paths, 0)
    if i==(len(ts)-2) :
        return np.mean(exerval)*np.exp(-r*ts[i+1])
    else :
        contval = beta[i+1,0]+beta[i+1,1]*Z
        V = np.maximum(exerval, contval)
        return np.mean(V)*np.exp(-r*ts[i+1])
    

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000   

# number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    for j in range(npaths):
        exerval,contval = np.maximum(K-paths[i,j],0) , betas_TVR[i, 0]+betas_TVR[i, 1]*Z[j] ## Using Estimates
        V[i, j] = np.maximum(exerval,contval)*np.exp(-r*ts[i])             ## Taking V[i,j] from estimates       
        EV[i,j] = nested_mc_TVR(paths[i, j], vol, r, q, i, ts, nnested,betas_TVR) # launch nested simulation to estimate E[V_{i+1}|F_i]

        
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Dual Price   (Upper Bound) = 5.1663


---

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Example 4 : </b> (<b>Bermudan-Asian Call Option</b>). In previous notebook, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> We reuse the exercise strategy derived from the basis functions used in previous notebook by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound.

In [12]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=np.float64)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
#np.mean(payoff * np.exp(-r * (ts[1] - ts[0])))
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[        nan,         nan],
       [ 0.24206701,  1.13272417],
       [-0.03384112,  1.11121262],
       [-0.19546972,  1.08942779],
       [-0.28952533,  1.07278118],
       [-0.35945773,  1.06053357],
       [-0.38594565,  1.0505587 ],
       [-0.39300888,  1.04156245],
       [-0.37079416,  1.03424445],
       [-0.32240829,  1.02564166],
       [-0.25505935,  1.01730625],
       [-0.16921958,  1.00809988],
       [        nan,         nan]])

In [15]:
def exer_or_cont_BA(i,A,K,Z,beta):
    contval = beta[i,0] + beta[i,1]*Z
    exerval = np.maximum(A-K,0) 
    return  exerval >= contval

exer_func_call= lambda S: np.maximum(S-K, 0)

In [16]:
def nested_mc_BA(S,A,vol, r, q, i, ts, nnested,betas_AC):
    nested_paths = np.full(nnested, S, dtype=np.float)
    nested_A     = A
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        nested_A     = (nested_A*(j-1)+nested_paths)/j
        Z            = (j*nested_A+(12-j)*nested_paths)/12
        exer_vals =  exer_func_call(nested_A)
        if j < len(ts)-1:
            bs_call = blackscholes_price(K, ts[-1]-ts[j], S=Z,vol=vol_, callput='call')
            ind = exer_or_cont_BA(j,nested_A,K,bs_call,betas_AC)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_A = nested_A[~ind]
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths


V0 = nested_mc_BA(S0,S0, vol, r, q, 0, ts, 1000000, betas_AC) 


npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float64)                 # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float64)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0  # at time 0, option value = continuation value

paths_A = np.vstack((paths[0], np.cumsum(paths[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages

for i in range(1, len(ts)-1):
    exer_vals =  exer_func_call(paths_A[i]) 
    Z=(i*paths_A[i]+(12-i)*paths[i])/12
    bs_call = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    for j in range(npaths):
        if exer_or_cont_BA(i, paths_A[i, j], K,bs_call[j], betas_AC):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_BA(paths[i, j], paths_A[i, j], vol, r, q, i, ts, nnested, betas_AC) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_BA(paths[i, j], paths_A[i, j], vol, r, q, i, ts, nnested, betas_AC)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func_call(paths_A[-1])*np.exp(-r*ts[-1])                         # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float64)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)

upper_bound = np.mean(np.amax(exer_func_call(paths_A[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(upper_bound))

duality_gap = upper_bound - V0

Primal Price (Lower Bound) = 5.3147
Dual Price   (Upper Bound) = 5.4069


<b>(b).</b> As an alternative set of basis functions at time $t_n$, we use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

In [17]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 5), np.nan, dtype=np.float64)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A = np.vstack((np.ones_like(pathsA[i]), pathsA[i],pathsS[i],pathsA[i]**2,pathsS[i]**2)).T  ## Using the New Basis 
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
#np.mean(payoff * np.exp(-r * (ts[1] - ts[0])))
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[            nan,             nan,             nan,
                    nan,             nan],
       [ 1.14821679e+02, -1.35871360e+00, -1.35871360e+00,
         8.06283441e-03,  8.06283371e-03],
       [ 1.15952887e+02, -7.83230744e-01, -1.95672861e+00,
         4.31519983e-03,  1.18800470e-02],
       [ 1.16641293e+02, -9.80012391e-01, -1.77176890e+00,
         5.46080855e-03,  1.07431910e-02],
       [ 1.21321381e+02, -1.48218077e+00, -1.36552148e+00,
         8.20794121e-03,  8.45927425e-03],
       [ 1.17809726e+02, -1.56628376e+00, -1.21030707e+00,
         8.77875142e-03,  7.50471245e-03],
       [ 1.20404455e+02, -1.86719768e+00, -9.62564819e-01,
         1.04796971e-02,  6.05619577e-03],
       [ 1.20330577e+02, -2.07171787e+00, -7.57418356e-01,
         1.17028576e-02,  4.81968956e-03],
       [ 1.19705865e+02, -2.28110808e+00, -5.35365424e-01,
         1.29487781e-02,  3.50229165e-03],
       [ 1.18436017e+02, -2.44288969e+00, -3.48357146e-01,
         1.39587427e-02

In [18]:
def exer_or_cont_BA_call(i,A,K,Z,beta):
    contval = sum([beta[i,m]*Z[m] for m in range(5)])
    exerval = np.maximum(A-K,0) 
    return  exerval >= contval

exer_func_call= lambda S: np.maximum(S-K, 0)

In [19]:
def nested_mc_BA(S,A,vol, r, q, i, ts, nnested,betas_AC):
    nested_paths = np.full(nnested, S, dtype=np.float)
    nested_A     = A
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        nested_A     = (nested_A*(j-1)+nested_paths)/j
        Z            = np.vstack((np.ones_like(nested_paths), nested_A,nested_paths,nested_A**2,nested_paths**2)) # New Basis
        exer_vals =  exer_func_call(nested_A)
        if j < len(ts)-1:
            ind = exer_or_cont_BA_call(j,nested_A,K,Z,betas_AC)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_A = nested_A[~ind]
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths


V0 = nested_mc_BA(S0,S0, vol, r, q, 0, ts, 1000000, betas_AC)  

npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float64)                 # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float64)                # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0  # at time 0, option value = continuation value

paths_A = np.vstack((paths[0], np.cumsum(paths[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages

for i in range(1, len(ts)-1):
    exer_vals =  exer_func_call(paths_A[i]) 
    Z=np.vstack((np.ones_like(paths[i]), paths_A[i],paths[i],paths_A[i]**2,paths[i]**2))
    for j in range(npaths):
        if exer_or_cont_BA_call(i, paths_A[i, j], K,Z[:,j], betas_AC):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_BA(paths[i, j], paths_A[i, j], vol, r, q, i, ts, nnested, betas_AC) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_BA(paths[i, j], paths_A[i, j], vol, r, q, i, ts, nnested, betas_AC)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func_call(paths_A[-1])*np.exp(-r*ts[-1])                         # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float64)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)

upper_bound = np.mean(np.amax(exer_func_call(paths_A[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(upper_bound))

duality_gab_newBasis = upper_bound-V0


Primal Price (Lower Bound) = 5.0285
Dual Price   (Upper Bound) = 5.4358


<b>(c).</b> We compare the numerical duality gaps obtained from (a) and (b).

In [21]:
print("Daulity Gap in Part a :", duality_gap)
print("Daulity Gap in Part b :", duality_gab_newBasis)

Daulity Gap in Part a : 0.09219424486599692
Daulity Gap in Part b : 0.40731952987251496


We observe that the part (a) gives us better results in terms of estimating the optimal exercise strategy because it has low duality gap 

---

<b style="color:darkorange">Example 5 : </b> (<b>Multiple Exercises</b>) Consider a chooser put option that allows its holder to exercise the option up to 3 times among 12 possible monthly exercise dates
$t_1=\frac{1}{12}, t_2=\frac{2}{12}, \cdots, t_{12}=1$.
If exercised at time $t_i$, the option holder is paid $\left(K-S_{t_i}\right)^+$ with $K=100$. No repetitive exercise at the same exercise date is allowed.
The underlying security price is assumed to follow the geometric Brownian motion

$$\frac{dS_t}{S_t}=(r-q)dt+\sigma dW_t$$

with $S_0=100$, $r=0.06$, $q=0.02$ and $\sigma=0.2$. Note that the standard Bermudan put option can be viewed as a special chooser put option with only one exercise right. 

<b>(a)</b> We modify the Longstaff-Schwartz algorithm for Bermudan option to price this chooser option and use quadratic polynomial basis function in regression : 
<ul>
<li> To improve the accuracy, the regression should be done only in the region where option is in the money
<li> After we have obtained estimations of continuation values from the Longstaff-Schwartz algorithm, we also run an independent Monte Carlo simulation to obtain a low-biased price.
</ul>

In [22]:
## Calculating the values for V3

S0, K, vol, r, q = 100, 100, 0.2, 0.06, 0.02
ts = np.linspace(0, 1, 13)
npaths = 1000000
paths = blackscholes_mc(S0, vol, r, q, ts, npaths)
n_exer = 3

V = np.full((n_exer+1, len(paths), len(paths[0])), 0, dtype=np.float64)  ## Creating matrix of option value 
V[0, :, :] = 0  # Value of the ption when 0 exercise opportunities left = 0
V[1:, -1, :] = np.maximum(K - paths[-1], 0)  # Value of option when just one exercise left on the expiry is the exercise value
CV = np.full((n_exer+1, len(paths), len(paths[0])), 0, dtype=np.float64) # Creating matrix of  continuation value
CV[0, :, :] = 0  # Continuation value when no exercise left is 0 

ind = np.full((n_exer, len(paths[0])), np.nan, dtype=bool) # matrix of decision indices across each path
poly_fit = np.full((n_exer, len(paths), 3), np.nan)  # polynomial fits across each time 

for i in range(len(ts) - 2, 0, -1):                        # backward induction
    idx = (paths[i] < K)
    discount = np.exp(-r * (ts[i + 1] - ts[i]))
    for exer_idx in range(n_exer):
        poly_fit[exer_idx, i, :] = np.polyfit(paths[i][idx], V[exer_idx + 1, i + 1, idx], deg=2)
        CV[exer_idx+1, i, :] = np.polyval(poly_fit[exer_idx, i, :], paths[i])
        CV[exer_idx+1, i, ~idx] = CV[exer_idx+1, i + 1, ~idx] * discount
    exerval = np.maximum(K - paths[i], 0)
    for exer_idx in range(1, n_exer+1):
        ind[exer_idx-1, :] = (exerval + CV[exer_idx-1, i, :]) > CV[exer_idx, i, :]
        V[exer_idx, i, ind[exer_idx-1, :]] = V[exer_idx-1, i + 1, ind[exer_idx-1, :]] * discount + exerval[ind[exer_idx-1, :]]
        V[exer_idx, i, ~ind[exer_idx-1, :]] = V[exer_idx, i + 1, ~ind[exer_idx-1, :]] * discount


print("Option Value (Estimation from Same Path) :",np.mean(V[n_exer, 1, :] * np.exp(-r * (ts[1] - ts[0]))))
print("Polynomial Estimates :",poly_fit)

Option Value (Estimation from Same Path) : 18.043787861527253
Polynomial Estimates : [[[            nan             nan             nan]
  [ 1.34898083e-02 -3.10818970e+00  1.82019882e+02]
  [ 1.41499514e-02 -3.24300766e+00  1.88672734e+02]
  [ 1.33366802e-02 -3.10415237e+00  1.82636525e+02]
  [ 1.21953975e-02 -2.90909023e+00  1.74205385e+02]
  [ 1.08246721e-02 -2.68005339e+00  1.64587141e+02]
  [ 9.84071035e-03 -2.52152279e+00  1.58166632e+02]
  [ 8.63231547e-03 -2.32687991e+00  1.50288913e+02]
  [ 7.36472392e-03 -2.12445841e+00  1.42199993e+02]
  [ 6.30893182e-03 -1.96383622e+00  1.36112357e+02]
  [ 5.03951945e-03 -1.77233094e+00  1.28959875e+02]
  [ 3.33482393e-03 -1.51291924e+00  1.19207564e+02]
  [            nan             nan             nan]]

 [[            nan             nan             nan]
  [ 2.69658275e-02 -6.21657221e+00  3.63928757e+02]
  [ 2.81751644e-02 -6.46782483e+00  3.76476930e+02]
  [ 2.63293628e-02 -6.14957659e+00  3.62515432e+02]
  [ 2.39021413e-02 -5.7339543

###### Estimating for Independent Paths

In [23]:
ind_paths = blackscholes_mc(S0, vol, r, q, ts, npaths)  ### These will be independent of previous pthas 

V = np.full((n_exer+1, len(ind_paths), len(ind_paths[0])), 0, dtype=np.float64)  ## Creating matrix of option value 
V[0, :, :] = 0  # Value of the ption when 0 exercise opportunities left = 0
V[1:, -1, :] = np.maximum(K - ind_paths[-1], 0)  # Value of option when just one exercise left on the expiry is the exercise value
CV = np.full((n_exer+1, len(ind_paths), len(ind_paths[0])), 0, dtype=np.float64) # Creating matrix of  continuation value
CV[0, :, :] = 0  # Continuation value when no exercise left is 0 
ind = np.full((n_exer, len(ind_paths[0])), np.nan, dtype=bool) # matrix of decision indices across each path

for i in range(len(ts) - 2, 0, -1):                        # backward induction
    idx = (ind_paths[i] < K)                               # worth exercise
    discount = np.exp(-r * (ts[i + 1] - ts[i]))
    for exer_idx in range(n_exer):
        CV[exer_idx+1, i, :] = np.polyval(poly_fit[exer_idx, i, :], ind_paths[i])
        CV[exer_idx+1, i, ~idx] = CV[exer_idx+1, i + 1, ~idx] * discount
    exerval = np.maximum(K - ind_paths[i], 0)
    for exer_idx in range(1, n_exer+1):
        ind[exer_idx-1, :] = (exerval + CV[exer_idx-1, i, :]) > CV[exer_idx, i, :]
        V[exer_idx, i, ind[exer_idx-1, :]] = V[exer_idx-1, i + 1, ind[exer_idx-1, :]] * discount + exerval[ind[exer_idx-1, :]]
        V[exer_idx, i, ~ind[exer_idx-1, :]] = V[exer_idx, i + 1, ~ind[exer_idx-1, :]] * discount


V3 = np.mean(V[n_exer, 1, :] * np.exp(-r * (ts[1] - ts[0])))        
        
print("Option Value (Estimation from Independent Paths) :",V3)


Option Value (Estimation from Independent Paths) : 18.041764743502128


<b>(b).</b> If $V_3$ be the price of the choose option and $V_1$ the price of the standard Bermudan put option with the same exercise schedule and strike. We want to compare $V_3$ with $3V_1$

In [24]:
payoff = np.maximum(K-paths[-1], 0)
poly_fit = deque([])
for i in range(len(ts)-2, 0, -1):
    idx=(paths[i]<K)
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    poly_fit.append(np.polyfit(paths[i][idx], payoff[idx], deg=2))
    contval = np.polyval(poly_fit[-1], paths[i])
    contval[~idx]=payoff[~idx]
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]

print("Polynomial Fits :")
pprint(poly_fit)

Polynomial Fits :
deque([array([ 3.31819143e-03, -1.50537353e+00,  1.18613014e+02]),
       array([ 5.04503986e-03, -1.76909312e+00,  1.28578182e+02]),
       array([ 6.31387520e-03, -1.96068673e+00,  1.35742809e+02]),
       array([ 7.37090403e-03, -2.12176422e+00,  1.41861069e+02]),
       array([ 8.68606783e-03, -2.33247862e+00,  1.50306816e+02]),
       array([ 9.87954473e-03, -2.52497902e+00,  1.58112356e+02]),
       array([ 1.09022068e-02, -2.69047309e+00,  1.64842505e+02]),
       array([ 1.21986307e-02, -2.90654627e+00,  1.73896791e+02]),
       array([ 1.33080427e-02, -3.09529723e+00,  1.82015381e+02]),
       array([ 1.41750064e-02, -3.24467178e+00,  1.88564348e+02]),
       array([ 1.33690555e-02, -3.08206276e+00,  1.80589318e+02])])


###### Estimating for Independent Paths

In [25]:
payoff = np.maximum(K-ind_paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    idx=(ind_paths[i]<K)
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    contval = np.polyval(poly_fit.popleft(), ind_paths[i])
    contval[~idx]=payoff[~idx]
    exerval = np.maximum(K-ind_paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]
V1 = np.mean(payoff*np.exp(-r*(ts[1]-ts[0])))

print("Bermudan Option Value V1 (single exercise) :",V1)

Bermudan Option Value V1 (single exercise) : 6.274608971627968


###### Comparing values of Bermudan Option Chooser Option with 3 Exercise and Normal Bermudan Option

In [26]:
print("Bermudan Chooser Option :",V3)
print("Normal Bermudan Option :",V1)
print("Comparing 3V1 and V3 :", "3V1=",3*V1,"V3=",V3,"V3>3V1=",V3>3*V1)

Bermudan Chooser Option : 18.041764743502128
Normal Bermudan Option : 6.274608971627968
Comparing 3V1 and V3 : 3V1= 18.823826914883902 V3= 18.041764743502128 V3>3V1= False


This is inline with our expectation because when we have 3 Bermudan Options we can exercise all the three at the same optimal stoppin time. When we have a single Bermudan Option with 3 Exercise Values we can exercise only once at an optimal stopping time whlie the other two exercise will be at some other time. This makes the value of 3 normal bermudan options greater than the price of a signle bermudan option with three exercise values.